In [2]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from utils.scraper import scrape_recipe,extract_nutrition
import re
import os

In [14]:
url = 'https://preppykitchen.com/wprm_print/cream-pie'
url = 'https://preppykitchen.com/buttermilk-pie'
# url = 'https://preppykitchen.com/cream-pie/'
url = "https://preppykitchen.com/apple-spice-cake/"
url = 'https://preppykitchen.com/chicken-sandwich/'
url= 'https://preppykitchen.com/apple-pie/'

try:
    # Send a GET request to the base page
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error fetching {url}: {e}")
    
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')


In [26]:
   
def extract_nutrition(soup):
    """Find all nutrition containers"""
    
    nutrition_elements = soup.select('.wprm-nutrition-label-text-nutrition-container')


    if not nutrition_elements :
        return None
    # Initialize an empty dictionary to store the nutrition data
    nutrition_dict = {}

    # Loop through each element and extract the nutrition details
    for element in nutrition_elements:
        # Extract the label (e.g., Calories, Carbohydrates)
        label = element.select_one('.wprm-nutrition-label-text-nutrition-label').text.strip(': ')
        
        # Extract the value (e.g., 443, 76)
        value = element.select_one('.wprm-nutrition-label-text-nutrition-value').text
        
        # Extract the unit (e.g., kcal, g, mg)
        unit = element.select_one('.wprm-nutrition-label-text-nutrition-unit').text
        
        # Store the data in the dictionary
        nutrition_dict[label] = (value,unit)
        
    return nutrition_dict   

extract_nutrition(soup)


{'Calories': ('443', 'kcal'),
 'Carbohydrates': ('76', 'g'),
 'Protein': ('5', 'g'),
 'Fat': ('15', 'g'),
 'Saturated Fat': ('6', 'g'),
 'Polyunsaturated Fat': ('2', 'g'),
 'Monounsaturated Fat': ('6', 'g'),
 'Trans Fat': ('0.1', 'g'),
 'Cholesterol': ('31', 'mg'),
 'Sodium': ('185', 'mg'),
 'Potassium': ('247', 'mg'),
 'Fiber': ('6', 'g'),
 'Sugar': ('43', 'g'),
 'Vitamin A': ('215', 'IU'),
 'Vitamin C': ('9', 'mg'),
 'Calcium': ('28', 'mg'),
 'Iron': ('2', 'mg')}

In [75]:

save_image(soup.find('div', class_='featured-image-class'), save_path='/recipe_images/featured/')

'/recipe_images/featured/Baileys-Shake-1.jpeg'

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
from datetime import datetime
import os
from fractions import Fraction

def convert_fraction_text(quantity_text):
    # Define a mapping from fraction symbols to numeric string equivalents
    fraction_map = {
        '¼': '0.25',  # Quarter
        '½': '0.5',   # Half
        '¾': '0.75',  # Three-quarters
        '⅓': '0.333', # One-third
        '⅔': '0.666', # Two-thirds
        '⅕': '0.2',   # One-fifth
        '⅖': '0.4',   # Two-fifths
        '⅗': '0.6',   # Three-fifths
        '⅘': '0.8',   # Four-fifths
        '⅙': '0.166', # One-sixth
        '⅚': '0.833', # Five-sixths
        '⅛': '0.125', # One-eighth
        '⅜': '0.375', # Three-eighths
        '⅝': '0.625', # Five-eighths
        '⅞': '0.875', # Seven-eighths
        '⅐': '0.142', # One-seventh
        '⅑': '0.111', # One-ninth
        '⅒': '0.1',   # One-tenth
    }

    # Check if the quantity text is in the mapping
    firstNumber = '0'
    if quantity_text[0] not in fraction_map:
        firstNumber = quantity_text[0]
    for number in quantity_text:
        if number in fraction_map:
            quantity_text =  firstNumber + fraction_map[number]
    
    #Treat examples like 1-2
    division = 1
    if '-' in quantity_text or 'to' in quantity_text :
        quantity_text = quantity_text.replace('-',' ')
        quantity_text = quantity_text.replace('to',' ')
        division = 2
    
    #Treat fractions 
    quantity_list = quantity_text.split()
    #We need to treat examples like '1 1/2'
    if(len(quantity_list) > 1):
        sum = 0
        for quantity in quantity_list:
            sum += Fraction(quantity)
        sum = sum/division
        quantity_text = str(sum)  
 
    return float(Fraction(quantity_text))

# Function to download the image and save it locally
import re
import os
import requests

import re
import os
import requests

def save_image(image, save_path='/recipe_images/card/'):
    """
    Save an image to the specified path.
    
    Parameters:
        image: BeautifulSoup tag containing the image
        save_path: Path to save the image
    
    Returns:
        Path to the saved image or None if the process fails
    """
    # Extract image URL
    try:
        image_url = image.find('img')['src']
    except (KeyError, TypeError) as e:
        print(f"\nError extracting image URL: {e}")
        return None
    
    # Handle 'data-lazy-src' as a fallback
    if not image_url or not any(ext in image_url for ext in ['.jpg', '.jpeg', '.png']):
        try:
            image_url = image.find('img')['data-lazy-src']
        except KeyError as e:
            print(f"\nError fetching lazy image URL: {e}")
            return None

    try:
        # Set headers for the request
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(image_url, headers=headers, stream=True)
        response.raise_for_status()

        # Match valid image extensions (.jpg, .jpeg, .png, etc.)
        match = re.search(r'([^/]+\.(jpg|jpeg|png))$', image_url, re.IGNORECASE)

        if match:
            image_name = match.group(1)  # Extract the image name with extension
        else:
            print("No valid image URL found.")
            return None

        if image_name:
            save_path = 'media' + os.path.join(save_path, image_name)
        
        # Ensure the directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        # Save the image
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        
        return save_path.removeprefix('media')  # Return relative path
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return None



def extract_time(time):
    """Convert time string to HH:MM:SS format"""
    if not time:
        return None
    
    time_string = ' '.join([x.text for x in time])
    
    hours = re.search(r'(\d+)\s*hours?', time_string, re.IGNORECASE)
    minutes = re.search(r'(\d+)\s*minutes?', time_string)
    seconds = re.search(r'(\d+)\s*seconds?', time_string)
    
    hours = int(hours.group(1)) if hours else 0
    minutes = int(minutes.group(1)) if minutes else 0
    seconds = int(seconds.group(1)) if seconds else 0
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


def extract_ingredients(soup):
    """Extract ingredients information with fractional quantities"""
    ingredients_list = []
    ingredients_container = soup.find('div', class_='wprm-recipe-ingredients-container')

    if ingredients_container:
        # ingredient_items = ingredients_container.find_all('li', class_='wprm-recipe-ingredient')
        groups_ingredients = ingredients_container.find_all('div', class_='wprm-recipe-ingredient-group')

        for group in groups_ingredients:
            groupname = group.find('h4', class_='wprm-recipe-group-name')
            ingredient_items = group.find_all('li', class_='wprm-recipe-ingredient')
            for item in ingredient_items:
                amount = item.find('span', class_='wprm-recipe-ingredient-amount')
                unit = item.find('span', class_='wprm-recipe-ingredient-unit')
                name = item.find('span', class_='wprm-recipe-ingredient-name')
                notes = item.find('span', class_='wprm-recipe-ingredient-notes')

                # # Handle fractional quantities (like ½, ¼)
                if amount:
                    quantity_text = amount.text.strip()
                    try:
                        # Convert the fraction to a float if it's a valid fraction
                        quantity = convert_fraction_text(quantity_text)
                        

                    except ValueError:
                        # If it's not a valid fraction (or not a number), keep it as None
                        print(f"it's not a valid fraction (or not a number), keep it as None")
                        quantity = None
                else:
                    quantity = None

                ingredient_dict = {
                    "ingredient": {
                        "name": name.text.strip() if name else None,
                        "nutrition": None
                    },
                    "quantity": quantity,
                    "unit": unit.text.strip() if unit else "whole",
                    "groupName" : groupname.text if groupname else None,
                    "notes": notes.text.strip() if notes else None
                }
                ingredients_list.append(ingredient_dict)

    return ingredients_list


def extract_instructions(soup):
    """Extract cooking instructions"""
    instructions_container = soup.find('div', class_='wprm-recipe-instructions-container')
    instructions = []
    
    groups_instructions = instructions_container.find_all('div', class_='wprm-recipe-instruction-group')

    for group in groups_instructions:
        groupname = group.find('h4', class_='wprm-recipe-group-name')
        instructions_group = group.find_all('ul', class_='wprm-recipe-instructions')
        
        if groupname :
            instructions.append(f"GroupName : {groupname.text}")
            for instruction in instructions_group:
                instruction_items = instruction.find_all('li', class_='wprm-recipe-instruction')
                # instructions = [f"{i+1}. {item.text.strip()}" for i, item in enumerate(instruction_items)]
                instructions.extend([f"{i+1}. {item.text.strip()}" for i, item in enumerate(instruction_items)])
        else :
            if instructions_container:
                instruction_items = instructions_container.find_all('li', class_='wprm-recipe-instruction')
                instructions = [f"{i+1}. {item.text.strip()}" for i, item in enumerate(instruction_items)]
    
    return "\n".join(instructions)


def extract_nutrition(soup):
    """Find all nutrition containers"""
    
    nutrition_elements = soup.select('.wprm-nutrition-label-text-nutrition-container')


    if not nutrition_elements :
        return None
    # Initialize an empty dictionary to store the nutrition data
    nutrition_dict = {}

    # Loop through each element and extract the nutrition details
    for element in nutrition_elements:
        # Extract the label (e.g., Calories, Carbohydrates)
        label = element.select_one('.wprm-nutrition-label-text-nutrition-label').text.strip(': ')
        
        # Extract the value (e.g., 443, 76)
        value = element.select_one('.wprm-nutrition-label-text-nutrition-value').text
        
        # Extract the unit (e.g., kcal, g, mg)
        unit = element.select_one('.wprm-nutrition-label-text-nutrition-unit').text
        
        # Store the data in the dictionary
        nutrition_dict[label] = f"{value} {unit}"
        
    return nutrition_dict    

def extract_tags(soup):
    tags_list = []
    tags_container = soup.find_all('meta', property='slick:category')
    if tags_container:
        for category in tags_container:
            tag = category['content'].split(':')[0]
            tags_list.append(tag)

    return tags_list

def extract_tags(soup):
    tags_list = []
    tags_container = soup.find_all('meta', property='slick:category')
    if tags_container:
        for category in tags_container:
            tag = category['content'].split(':')[0]
            tags_list.append(tag)

    return tags_list

def extract_cuisines(soup):
    cuisine_list = []
    cuisine_container = soup.find('span', class_='wprm-recipe-cuisine').text.strip().split(',')
    if cuisine_container:
        for cuisine in cuisine_container:
            cuisine = cuisine.replace(' ', '').capitalize()
            cuisine_list.append(cuisine)

    return cuisine_list

def extract_courses(soup):
    course_list = []
    course_container = soup.find('span', class_='wprm-recipe-course').text.strip().split(',')
    if course_container:
        for course in course_container:
            course = course.replace(' ', '').capitalize()
            course_list.append(course)

    return course_list

In [76]:
import re

url = "https://preppykitchen.com/wp-content/uploads/2024/12/Banana-Waffles-Recipe-Card-330x330.jpg"
match = re.search(r'([^/]+\.jpg)$', url)

if match:
    jpg_name = match.group(1)
    print(jpg_name)  # Output: Banana-Waffles-Recipe-Card-330x330.jpg
else:
    print("No match found")


Banana-Waffles-Recipe-Card-330x330.jpg


In [52]:
import json
import requests
from tqdm import tqdm  # Import tqdm for the progress bar

# Open and read the JSON file
with open('dataScraped/sitemapURLS.json', 'r') as file:
    data = json.load(file)


data['post-sitemap']

sitesParsed = 0
for site in data['post-sitemap'][1:10]:
    # match = re.search(r'([^/]+\.jpg)$', url)
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(site, headers=headers)
        response.raise_for_status()
        
        if response.url != 'https://preppykitchen.com':
            sitesParsed += 1

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
    


In [71]:
import json
import requests
from tqdm import tqdm  # Import tqdm for the progress bar

# Open and read the JSON file
with open('dataScraped/sitemapURLS.json', 'r') as file:
    data = json.load(file)

# Get the URLs to process
sites_to_process = data['post-sitemap'][1:10]

sites_parsed = 0

# Use tqdm to show the progress bar
print("Processing sites:")
for site in tqdm(sites_to_process, desc="Progress", unit=" site"):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(site, headers=headers)
        response.raise_for_status()

        if response.url != 'https://preppykitchen.com':
            sites_parsed += 1

    except requests.exceptions.RequestException as e:
        print(f"\nError fetching {site}: {e}")

print(f"\nFinished processing {len(sites_to_process)} sites.")
print(f"Successfully parsed {sites_parsed} sites.")

    


Processing sites:


Progress: 100%|██████████| 9/9 [00:00<00:00,  9.10 site/s]


Finished processing 9 sites.
Successfully parsed 9 sites.


In [104]:
import json
import requests
from tqdm import tqdm  # Import tqdm for the progress bar

with open('dataScraped/recipe.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    


{'recipes': [{'data': {'type': 'RecipeDetailAPIView',
    'id': -1,
    'attributes': {'title': 'Cute Cupcakes',
     'difficulty': None,
     'image_card': '/recipe_images/card/Shag-cupcakes-recipe-330x330.jpg',
     'image': '/recipe_images/featured/Shag-Cupcakes-Feature.jpg',
     'course': {'id': -1, 'name': 'Dessert', 'type': 'Course'},
     'cuisine': {'id': -1, 'name': 'American', 'type': 'Cuisine'},
     'prep_time': '00:15:00',
     'cook_time': '00:20:00',
     'cool_time': None,
     'total_time': '00:35:00',
     'created': None,
     'modified': None,
     'description': "Charming, and cheerful cupcakes that are perfect for any celebration! Inside they're perfect, moist vanilla cake and on top a shaggy mop of vanilla buttercream!",
     'status': 1,
     'activate_date': None,
     'deactivate_date': None,
     'servings': 18,
     'ingredients': [{'ingredient': {'name': 'all-purpose flour',
        'nutrition': None},
       'quantity': '1 2/3',
       'unit': 'cup',
    

In [107]:
path = 'media/recipe_images/card/Shag-cupcakes-recipe-330x330.jpg'

path.removeprefix('media')
print(path)

media/recipe_images/card/Shag-cupcakes-recipe-330x330.jpg
